In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pathlib import Path

from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.metrics import *

In [2]:
df_Ch001 = pd.DataFrame(pd.read_pickle(Path("C:/Users/Victor/Desktop/PIR/process/Ch001.pkl")))
df_Ch009 = pd.DataFrame(pd.read_pickle(Path("C:/Users/Victor/Desktop/PIR/process/Ch009.pkl")))
df_Ch020 = pd.DataFrame(pd.read_pickle(Path("C:/Users/Victor/Desktop/PIR/process/Ch020.pkl")))
df_Ch022 = pd.DataFrame(pd.read_pickle(Path("C:/Users/Victor/Desktop/PIR/process/Ch022.pkl")))

In [3]:
from model_util import data_scale
from sklearn.preprocessing import MinMaxScaler, StandardScaler

df_input=[df_Ch001, df_Ch009, df_Ch020]
df_input_scaled=[]

LOG_SCALE= False
scaler=MinMaxScaler()

for df in df_input:
    df_input_scaled.append(data_scale(df, scaler, LOG_SCALE))

df_output_scaled, output_scaler = data_scale(df_Ch022, scaler, LOG_SCALE, output= True)

In [4]:
size=len(df_Ch022.index)
split=int(size*0.15)

MAE, MSE, RMSE = [], [], []
df_pred = pd.DataFrame(0, index= df_Ch022.index[:split], columns=df_Ch022.columns)

In [5]:
p=15
min = output_scaler.data_min_[p]
max = output_scaler.data_max_[p]
print(min,max)

0.0 4573.0


In [13]:
#format data
data_scaled=np.dstack(df_input_scaled)
target_scaled=df_output_scaled.copy().values

for p in range(len(df_Ch022.columns)):

    #select data
    X_train_p = data_scaled[split:,p,:]
    y_train_p = target_scaled[split:,p]
    X_test_p = data_scaled[:split,p,:]
    y_test_p = target_scaled[:split,p]

    #gridsearch
    svr_grid = GridSearchCV(SVR(kernel='rbf', gamma=0.1),
                   param_grid = {"C": [1e0, 1e1, 1e2, 1e3],
                               "gamma": np.logspace(-2, 2, 5)})

    svr_grid.fit(X_train_p, y_train_p)
    C = svr_grid.best_params_['C']
    gamma = svr_grid.best_params_['gamma']

    #learning curve
    svr_lc = SVR(kernel='rbf', C=C, gamma=gamma)
    train_sizes, train_scores_svr, test_scores_svr = \
    learning_curve(svr_lc, X_train_p, y_train_p, train_sizes=np.linspace(0.1, 1, 10),
                   scoring="neg_mean_squared_error", cv=10)

    split2 = train_sizes[np.argmin(-test_scores_svr.mean(1))]
    if split2 >int(size*0.5):

        X_train_p = data_scaled[size-split2:,p,:]
        y_train_p = target_scaled[size-split2:,p]

    #calcul predict test, error
    svr=SVR(kernel='rbf', C=C, gamma=gamma)
    svr.fit(X_train_p, y_train_p)
    y_pred_test_p = svr.predict(X_test_p)

    min = output_scaler.data_min_[p]
    max = output_scaler.data_max_[p]
    y_test_p= (max-min)*y_test_p+min
    y_pred_test_p = (max-min)*y_pred_test_p+min

    MAE.append(mean_absolute_error(y_test_p,y_pred_test_p))
    MSE.append(mean_squared_error(y_test_p,y_pred_test_p))
    RMSE.append(mean_squared_error(y_test_p, y_pred_test_p, squared= False))

    df_pred[df_pred.columns[p]] = y_pred_test_p

In [14]:
df_pred.to_pickle(Path("C:/Users/Victor/Desktop/PIR/model/saved_models/SVM_predict_test.pkl"))
df_metrics=pd.DataFrame({"MAE": MAE, "MSE": MSE, "RMSE": RMSE})
df_metrics.to_pickle(Path("C:/Users/Victor/Desktop/PIR/model/saved_models/SVM_metrics_test.pkl"))

In [15]:
df_metrics

,MAE,MSE,RMSE
0,10.952686,1.250180e+02,11.181146
1,20.183472,4.087399e+02,20.217317
2,59015.891803,4.000360e+09,63248.401681
3,24.877560,6.254434e+02,25.008867
4,4.768083,3.533907e+01,5.944667
5,3.069253,2.128084e+01,4.613116
6,3.596462,2.472355e+01,4.972278
7,4.656917,2.904815e+01,5.389634
8,222.424257,5.021693e+04,224.091344
9,6.305944,8.254278e+01,9.085306
